<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_5_2___RAG_router.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -qU llama-index  llama-index-embeddings-huggingface llama-index-llms-groq

Ten kod instaluje trzy pakiety Pythona związane z LlamaIndex:

- `llama-index` - główna biblioteka do budowania indeksów i przetwarzania danych
- `llama-index-embeddings-huggingface` - moduł do generowania embedingów za pomocą modeli HuggingFace
- `llama-index-llms-groq` - integracja z modelami językowymi Groq

Flaga `-q` wycisza komunikaty instalacji, a `-U` aktualizuje pakiety do najnowszych wersji.

In [ ]:
import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
from llama_index.core import Settings, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
import nest_asyncio
from google.colab import userdata
from pprint import pp
nest_asyncio.apply()

Ten kod importuje niezbędne moduły i komponenty:

`os` - standardowa biblioteka Pythona do operacji systemowych

Z pakietu llama-index.core:
- `SimpleDirectoryReader` - czyta dokumenty z katalogu
- `SentenceSplitter` - dzieli tekst na zdania
- `Settings` - ustawienia globalne
- `VectorStoreIndex` - indeks wektorowy do przechowywania dokumentów
- `SummaryIndex` - indeks do tworzenia streszczeń
- `QueryEngineTool` - narzędzie do przetwarzania zapytań
- `RouterQueryEngine` - silnik kierujący zapytania
- `LLMSingleSelector` - selektor modelu językowego

Dodatkowo:
- `Groq` - integracja z modelami Groq
- `HuggingFaceEmbedding` - embedingi z HuggingFace
- `nest_asyncio` - obsługa zagnieżdżonych pętli asynchronicznych
- `userdata` - dostęp do danych użytkownika w Colab
- `pp` - funkcja do ładnego wyświetlania struktur danych

`nest_asyncio.apply()` włącza obsługę zagnieżdżonych pętli asynchronicznych.

In [ ]:
class CFG:
  model1 = "llama3-8b-8192"
  model2 = "BAAI/bge-small-en-v1.5"
  temperature = 0.1
  chunksize = 1024

In [ ]:
os.environ['GROQ_API_KEY'] = userdata.get('groq')

# Data and model

In [ ]:
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2024-11-28 11:31:35--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  8.78MB/s    in 1.8s    

2024-11-28 11:31:38 (8.78 MB/s) - ‘metagpt.pdf’ saved [16911937/16911937]



To polecenie pobiera dokument PDF z OpenReview.

`wget` - narzędzie do pobierania plików z internetu
`"https://openreview.net/pdf?id=VtmBAGCN7o"` - adres URL pliku PDF
`-O metagpt.pdf` - zapisuje pobrany plik pod nazwą "metagpt.pdf"

In [ ]:

documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

Ta linia kodu wczytuje zawartość pliku PDF:

`SimpleDirectoryReader` to klasa z LlamaIndex służąca do odczytu dokumentów. Dostaje parametr `input_files` z listą zawierającą nazwę pliku "metagpt.pdf". Metoda `load_data()` przetwarza plik i ładuje jego zawartość do zmiennej `documents`.

Ten kod wykonuje automatyczną ekstrakcję tekstu z PDF-a, co pozwala na dalsze przetwarzanie jego zawartości w formie tekstowej.

In [ ]:

llm = Groq(model = CFG.model1)


Settings.llm = llm

Ten kod konfiguruje model językowy:

`llm = Groq(model = CFG.model1)` - tworzy instancję modelu Groq, używając modelu zdefiniowanego w zmiennej konfiguracyjnej `CFG.model1`

`Settings.llm = llm` - ustawia utworzony model jako domyślny model językowy w globalnych ustawieniach LlamaIndex. Dzięki temu wszystkie komponenty będą korzystać z tego samego modelu.

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name = CFG.model2
)

Ta linia kodu konfiguruje model do tworzenia embedingów (reprezentacji wektorowych tekstu):

`Settings.embed_model` - ustawia globalny model do embedingów w LlamaIndex

Używa się tu `HuggingFaceEmbedding` - implementacji embedingów z biblioteki HuggingFace. Konstruktor przyjmuje parametr `model_name` wskazujący na konkretny model zdefiniowany w zmiennej konfiguracyjnej `CFG.model2`.

Takie embedingi będą używane do przekształcania tekstu w wektory liczbowe, co pozwala na wyszukiwanie podobieństw między fragmentami tekstu i efektywne przetwarzanie dokumentów.

# Narzędzia

In [ ]:

splitter = SentenceSplitter(chunk_size = CFG.chunksize)


Ta linia tworzy obiekt do dzielenia tekstu na mniejsze fragmenty:

`SentenceSplitter` to narzędzie, które dzieli tekst na zdania lub fragmenty. Parametr `chunk_size`, ustawiony na wartość z `CFG.chunksize`, określa maksymalną długość pojedynczego fragmentu tekstu.

Taki podział tekstu na mniejsze części jest kluczowy dla efektywnego przetwarzania dokumentów, gdyż modele językowe i wektorowe mają ograniczenia co do długości tekstu, który mogą przetworzyć na raz.

In [ ]:
nodes = splitter.get_nodes_from_documents(documents)

Ta linia wykonuje podział dokumentów na mniejsze fragmenty tekstowe:

`splitter.get_nodes_from_documents(documents)` bierze wcześniej wczytane dokumenty i dzieli je na "węzły" (nodes) według wcześniej zdefiniowanych parametrów podziału.

Każdy węzeł reprezentuje fragment tekstu o określonej długości (zgodnej z ustawionym `chunk_size`). Węzły zawierają nie tylko tekst, ale również metadane jak pozycja w oryginalnym dokumencie czy relacje z innymi węzłami.

Te podzielone fragmenty są zapisywane do zmiennej `nodes` i będą podstawą do budowania indeksu wektorowego.

In [ ]:

summary_index = SummaryIndex(nodes)

Ta linia tworzy indeks streszczeń z wcześniej przygotowanych węzłów tekstowych:

`SummaryIndex` to specjalny typ indeksu w LlamaIndex, który jest zoptymalizowany do generowania streszczeń dokumentów. Przyjmuje jako argument `nodes` - wcześniej utworzone fragmenty tekstu.

Ten indeks pozwala na generowanie streszczeń całego dokumentu lub jego wybranych części, zachowując przy tym istotne informacje z oryginalnego tekstu.

In [ ]:

vector_index = VectorStoreIndex(nodes)

Ta linia tworzy indeks wektorowy z przygotowanych wcześniej węzłów tekstowych:

`VectorStoreIndex` zamienia fragmenty tekstu na wektory liczbowe (używając wcześniej skonfigurowanego modelu embedingów) i przechowuje je w strukturze umożliwiającej szybkie wyszukiwanie podobnych fragmentów.

Ten indeks jest kluczowy dla efektywnego wyszukiwania semantycznego - pozwala znajdować fragmenty tekstu podobne znaczeniowo do zadanego zapytania, nawet jeśli nie zawierają dokładnie tych samych słów.

In [ ]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

Ta linia tworzy silnik zapytań do indeksu streszczeń:

`summary_index.as_query_engine()` zamienia indeks streszczeń na silnik zapytań, który pozwala na interakcję z zawartością dokumentów. Przyjmuje dwa parametry:

- `response_mode="tree_summarize"` - używa hierarchicznego podejścia do tworzenia streszczeń, gdzie tekst jest organizowany w strukturę drzewiastą w celu lepszego zachowania kontekstu
- `use_async=True` - włącza tryb asynchroniczny, co pozwala na równoległe przetwarzanie i lepszą wydajność

Utworzony silnik zapytań jest zapisywany do zmiennej `summary_query_engine` i będzie służył do generowania streszczeń na podstawie zapytań.

In [ ]:

vector_query_engine = vector_index.as_query_engine()

In [ ]:
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

Ta linia tworzy silnik zapytań do indeksu wektorowego:

`vector_index.as_query_engine()` przekształca indeks wektorowy w silnik zapytań, który umożliwia wyszukiwanie semantyczne w dokumentach.

Ten silnik zapytań będzie używał reprezentacji wektorowych tekstu do znajdowania fragmentów dokumentów najbardziej odpowiadających zadanemu pytaniu, wykorzystując podobieństwo wektorów do określania trafności odpowiedzi.

In [ ]:

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

Ta linia tworzy narzędzie do obsługi zapytań bazujące na silniku wektorowym:

`QueryEngineTool.from_defaults()` tworzy narzędzie z dwoma parametrami:

- `query_engine=vector_query_engine` - wskazuje, który silnik zapytań ma być używany (w tym przypadku silnik wektorowy)
- `description` - zawiera opis przeznaczenia narzędzia: "Useful for retrieving specific context from the MetaGPT paper" (Przydatne do wyszukiwania konkretnych informacji z artykułu o MetaGPT)

To narzędzie będzie używane do wyszukiwania konkretnych fragmentów tekstu z dokumentu na podstawie zapytań użytkownika.

In [ ]:

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

# można zawinąć!

Ten kod tworzy zaawansowany silnik zapytań, który potrafi kierować pytania do odpowiednich narzędzi:

`RouterQueryEngine` to router zapytań, który otrzymuje trzy parametry:

- `selector=LLMSingleSelector.from_defaults()` - selektor wykorzystujący model językowy do wyboru najlepszego narzędzia dla danego zapytania

- `query_engine_tools` - lista dostępnych narzędzi do przetwarzania zapytań:
  - `summary_tool` - do generowania streszczeń
  - `vector_tool` - do wyszukiwania konkretnych informacji

- `verbose=True` - włącza szczegółowe logowanie, co pozwala śledzić proces wyboru i przetwarzania zapytań

Ten router automatycznie wybiera najlepsze narzędzie do obsługi konkretnego zapytania - albo generowanie streszczenia, albo wyszukiwanie szczegółowych informacji.

Komentarz "można zawinąć!" oznacza, że kod jest gotowy do użycia i można rozpocząć zadawanie pytań do dokumentu.

# Test

In [ ]:
response = query_engine.query("what is this document about?")
pp(str(response))

Selecting query engine 0: The question is asking about the purpose of the document, and the options suggest that it is related to summarization, which is more relevant to the first option..
('A meta-programming framework called MetaGPT, designed for multi-agent '
 'collaboration based on Large Language Models (LLMs), which aims to improve '
 'the efficiency and effectiveness of LLM-based multi-agent systems by '
 'incorporating Standard Operating Procedures (SOPs) and structured '
 'communication interfaces.')


In [ ]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
pp(str(response))

Selecting query engine 0: The question is about summarization, which is related to MetaGPT, making choice 1 the most relevant..
('Through a shared message pool, where they can publish structured messages '
 'and access messages from other entities transparently.')


In [ ]:
response = query_engine.query(
    "How do agents share information with other agents? Do not summarize"
)
pp(str(response))

Selecting query engine 1: The question is asking about retrieving specific context from the MetaGPT paper, which suggests that the question is focused on the technical details of the paper rather than summarization..
('Agents share information with other agents through a shared message pool, '
 'where they publish structured messages and can also subscribe to relevant '
 'messages based on their profiles.')


In [ ]:
response = query_engine.query(
    "How do agents share information with other agents? Explain in detail"
)
pp(str(response))

Selecting query engine 0: The question is asking for a detailed explanation, which is more suitable for summarization questions, making choice 1 the most relevant..
